LSTM with attention.

Got mse 173ish with all agent data


In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2025-05-30 03:13:38.189181: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-30 03:13:38.189251: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-30 03:13:38.190869: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-30 03:13:38.201132: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# load data
import numpy as np
train_file = np.load('data/train.npz')
train_data = train_file['data']
print("train_data's shape", train_data.shape)
test_file = np.load('data/test_input.npz')
test_data = test_file['data']
print("test_data's shape", test_data.shape)


train_data's shape (10000, 50, 110, 6)
test_data's shape (2100, 50, 50, 6)


In [3]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense, Input, RepeatVector, TimeDistributed, Dropout

In [4]:
def standardize_data_dimensions(scenario_data):
    """
    Standardize position data by centering a single scenario at the origin.
    
    :param scenario_data: numpy array of shape (50, 110, 6)
                         where dimensions are [position_x, position_y, velocity_x, velocity_y, heading, object_type]
    :returns: tuple of (standardized_data, min_values)
             - standardized_data: same shape as input with centered positions
             - min_values: array of shape (2,) containing [min_x, min_y] for this scenario
    """
    # Copy the data to avoid modifying the original
    standardized_data = scenario_data.copy()
    
    # Extract position data (first 2 dimensions)
    positions = scenario_data[:, :, :2]  # Shape: (50, 110, 2)
    
    # Create mask for non-zero positions (to ignore padding)
    # We consider a position valid if it's not (0,0) or if the object_type is not 0
    object_types = scenario_data[:, :, 5]  # Shape: (50, 110)
    valid_mask = (positions[:, :, 0] != 0) | (positions[:, :, 1] != 0) | (object_types != 0)
    
    # Find min values across all valid positions in this scenario
    if np.any(valid_mask):
        valid_positions = positions[valid_mask]  # Shape: (num_valid_points, 2)
        min_x = np.min(valid_positions[:, 0])
        min_y = np.min(valid_positions[:, 1])
    else:
        # If no valid positions found, use 0 as min values
        min_x = 0
        min_y = 0
    
    # Store min values
    min_values = np.array([min_x, min_y])
    
    # Standardize positions by subtracting min values
    # Only modify non-zero positions to preserve padding
    for agent_idx in range(scenario_data.shape[0]):
        for time_idx in range(scenario_data.shape[1]):
            if valid_mask[agent_idx, time_idx]:
                standardized_data[agent_idx, time_idx, 0] -= min_x  # position_x
                standardized_data[agent_idx, time_idx, 1] -= min_y  # position_y
    
    return standardized_data, min_values


def denormalize_predictions(predictions, min_values):
    """
    Helper function to add back the min values to predicted positions.
    
    :param predictions: predicted data with standardized positions, shape (50, 110, 6) or similar
    :param min_values: array of shape (2,) containing [min_x, min_y] for this scenario
    :returns: predictions with original coordinate system restored
    """
    denormalized = predictions.copy()
    
    # Add back the min values to restore original coordinate system
    # Assuming predictions have position_x and position_y as first two dimensions
    denormalized[:, :, 0] += min_values[0]  # position_x
    denormalized[:, :, 1] += min_values[1]  # position_y
    
    return denormalized

In [5]:
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
from tqdm import tqdm

def standardize_single_scene(scene_data):
    """
    Wrapper function to standardize a single scene and return both standardized data and min values.
    This function will be called in parallel.
    """
    standardized_scene, min_vals = standardize_data_dimensions(scene_data)
    return standardized_scene, min_vals

def parallel_standardize_training_data(train_data, n_jobs=-1, verbose=True):
    """
    Parallelize the standardization of training data across all scenes.
    
    :param train_data: numpy array of shape (10000, 50, 110, 6)
    :param n_jobs: number of parallel jobs (-1 uses all available cores)
    :param verbose: whether to show progress bar
    :returns: tuple of (standardized_data, min_values_array)
    """
    print(f"Standardizing {train_data.shape[0]} scenes using {multiprocessing.cpu_count() if n_jobs == -1 else n_jobs} cores...")
    
    # Use joblib to parallelize the processing
    if verbose:
        # With progress bar
        results = Parallel(n_jobs=n_jobs)(
            delayed(standardize_single_scene)(train_data[i]) 
            for i in tqdm(range(train_data.shape[0]), desc="Processing scenes")
        )
    else:
        # Without progress bar
        results = Parallel(n_jobs=n_jobs)(
            delayed(standardize_single_scene)(train_data[i]) 
            for i in range(train_data.shape[0])
        )
    
    # Unpack results
    standardized_scenes, min_values_list = zip(*results)
    
    # Convert to numpy arrays
    standardized_data = np.array(standardized_scenes)
    min_values_array = np.array(min_values_list)
    
    print(f"Standardization complete!")
    print(f"Standardized data shape: {standardized_data.shape}")
    print(f"Min values shape: {min_values_array.shape}")
    
    return standardized_data, min_values_array

In [6]:
standardized_train_data, min_values = parallel_standardize_training_data(
    train_data, 
    n_jobs=-1,  # Use all available cores
    verbose=True
)
standardized_train_data.shape

Standardizing 10000 scenes using 40 cores...


Processing scenes: 100%|██████████| 10000/10000 [00:15<00:00, 650.39it/s]


Standardization complete!
Standardized data shape: (10000, 50, 110, 6)
Min values shape: (10000, 2)


(10000, 50, 110, 6)

In [7]:
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, RepeatVector, TimeDistributed, 
    Concatenate, Activation, Dot, Layer, BatchNormalization, 
    LayerNormalization, Add
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import Orthogonal, GlorotUniform
from tensorflow.keras.regularizers import l2
import tensorflow.keras.backend as K

class ScaleLayer(Layer):
    def __init__(self, scale_factor, **kwargs):
        super().__init__(**kwargs)
        self.scale_factor = scale_factor
    
    def call(self, x):
        return x / self.scale_factor

class MaxSubtractLayer(Layer):
    def call(self, x):
        return x - K.max(x, axis=-1, keepdims=True)



In [8]:
from tensorflow import keras

def load_model(model_path='lstm2.keras'):
    
    custom_objects = {
        'ScaleLayer':ScaleLayer,
        'MaxSubtractLayer':MaxSubtractLayer
    }
    
    model = keras.models.load_model(model_path, custom_objects=custom_objects, safe_mode=False)
    print(f"Keras model loaded from {model_path}")
    return model


In [9]:
model = load_model()

2025-05-30 03:14:04.842669: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10534 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1


Keras model loaded from lstm2.keras


In [9]:
from tensorflow.keras.layers import Input, LSTM, Dense, RepeatVector, TimeDistributed, Dropout, Attention, Concatenate, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

def create_lstm_encoder_decoder_with_attention(input_dim, output_dim, timesteps_in, timesteps_out,
                                             lstm_units=512, num_layers=3, loss_fn='mse', lr=0.001):
    inputs = Input(shape=(timesteps_in, input_dim))

    # Encoder - preserve all timesteps for attention
    x = inputs
    for i in range(num_layers):
        x = LSTM(lstm_units, return_sequences=True, name=f'encoder_lstm_{i}')(x)
    encoder_outputs = x  # All encoder hidden states
    
    # Get context vector from final timestep
    context_vector = Lambda(lambda x: x[:, -1, :])(encoder_outputs)

    # Decoder
    decoder_input = RepeatVector(timesteps_out)(context_vector)
    
    # Decoder LSTM layers
    decoder_output = decoder_input
    for i in range(num_layers):
        decoder_output = LSTM(lstm_units, return_sequences=True, name=f'decoder_lstm_{i}')(decoder_output)
    
    # Attention mechanism
    attention_layer = Attention(name='attention')
    attended_context = attention_layer([decoder_output, encoder_outputs])
    
    # Combine decoder output with attended context
    combined = Concatenate(axis=-1)([decoder_output, attended_context])
    
    # Output layers
    x = TimeDistributed(Dense(128, activation='relu'))(combined)
    x = TimeDistributed(Dense(64, activation='relu'))(x)
    outputs = TimeDistributed(Dense(output_dim))(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=lr, clipnorm=0.5), loss='mse', metrics=['mae'])

    return model

In [10]:
from keras.src.callbacks import LearningRateScheduler, EarlyStopping, Callback
from keras.src.optimizers import Adam
from keras import Model
import numpy as np


from tensorflow.keras.callbacks import Callback

class DynamicReduceLROnPlateau(Callback):
    def __init__(self, factor=0.1, patience=3, min_lr=1e-7, verbose=1):
        super().__init__()
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.verbose = verbose
        self.best_val_loss = float('inf')
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current_val_loss = logs.get("val_loss")

        if current_val_loss is None:
            return  # can't do anything if val_loss isn't available

        if current_val_loss < self.best_val_loss - 1e-4:  # a small delta
            self.best_val_loss = current_val_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                old_lr = float(self.model.optimizer.learning_rate.numpy())
                if old_lr > self.min_lr:
                    new_lr = max(old_lr * self.factor, self.min_lr)
                    self.model.optimizer.learning_rate.assign(new_lr)
                    if self.verbose:
                        print(f"\nEpoch {epoch+1}: val_loss did not improve. Reducing LR from {old_lr:.6f} to {new_lr:.6f}")
                    self.wait = 0  # reset after LR reduction

In [11]:
class LRThresholdCallback(Callback):
    def __init__(self, threshold=9e-5):
        super().__init__()
        self.threshold = threshold
        self.should_stop = False

    def on_epoch_end(self, epoch, logs=None):
        lr = float(self.model.optimizer.learning_rate.numpy())
        if lr < self.threshold:
            print(f"\nLearning rate {lr:.6f} < threshold {self.threshold}, moving to Phase 2.")
            self.model.stop_training = True


In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

class GradientMonitoringCallback(Callback):
    def __init__(self, clip_min=1e-4, clip_max=1e2, monitor_frequency=3):
        """
        Monitor gradient norms during training
        
        Args:
            clip_min: Minimum threshold for gradient norms
            clip_max: Maximum threshold for gradient norms  
            monitor_frequency: How often to check gradients (every N batches)
        """
        print(f"🔧 GradientMonitoringCallback initialized with clip_min={clip_min}, clip_max={clip_max}, monitor_freq={monitor_frequency}")
        self.clip_min = clip_min
        self.clip_max = clip_max
        self.monitor_frequency = monitor_frequency
        self.batch_count = 0
        self.total_calls = 0
        self.gradient_checks = 0
        self.fallback_calls = 0
        
    def on_train_begin(self, logs=None):
        print("🚀 GradientMonitoringCallback: Training started!")
        self.batch_count = 0
        self.total_calls = 0
        self.gradient_checks = 0
        self.fallback_calls = 0
        
    # def on_epoch_begin(self, epoch, logs=None):
    #     print(f"📍 GradientMonitoringCallback: Starting epoch {epoch + 1}")
        
    # def on_train_batch_begin(self, batch, logs=None):
    #     # Just to prove we're being called
    #     if batch % 50 == 0:  # Print every 50 batches to avoid spam
    #         print(f"⚡ GradientMonitoringCallback: Batch {batch} starting")
        
    def on_train_batch_end(self, batch, logs=None):
        self.batch_count += 1
        self.total_calls += 1
        
        # Print every time to show we're being called
        # if batch % 50 == 0:  # Print every 50 batches
            # print(f"📊 GradientMonitoringCallback: Batch {batch} ended (total calls: {self.total_calls})")
        
        # Only monitor every N batches to avoid performance overhead
        if self.batch_count % self.monitor_frequency != 0:
            return
            
        # print(f"🔍 GradientMonitoringCallback: Checking gradients at batch {batch} (check #{self.gradient_checks + 1})")
        
        # Get gradients from the optimizer's current state
        try:
            # Access the model's optimizer to get gradient information
            optimizer = self.model.optimizer
            print(f"   📋 Optimizer type: {type(optimizer).__name__}")
            
            # Get trainable variables
            trainable_vars = self.model.trainable_variables
            print(f"   📈 Number of trainable variables: {len(trainable_vars)}")
            
            if hasattr(optimizer, 'get_gradients'):
                print("   ✅ Optimizer has get_gradients method")
                # For some optimizers, we can access gradients directly
                grads = optimizer.get_gradients(self.model.total_loss, trainable_vars)
                print(f"   📊 Retrieved {len([g for g in grads if g is not None])} gradients")
            else:
                print("   ❌ Optimizer doesn't have get_gradients, using variable norms")
                # Alternative approach: check the current variable states
                grad_norms = []
                for i, var in enumerate(trainable_vars):
                    if var is not None:
                        var_norm = tf.norm(var)
                        grad_norms.append(var_norm)
                        if i < 3:  # Print first 3 for debugging
                            print(f"      Variable {i} norm: {float(var_norm.numpy()):.2e}")
                
                self._check_norms(grad_norms, "Variable")
                self.gradient_checks += 1
                return
                
            # Compute gradient norms
            grad_norms = []
            for i, grad in enumerate(grads):
                if grad is not None:
                    grad_norm = tf.norm(grad)
                    grad_norms.append(grad_norm)
                    if i < 3:  # Print first 3 for debugging
                        print(f"      Gradient {i} norm: {float(grad_norm.numpy()):.2e}")
                    
            print(f"   ✅ Computed {len(grad_norms)} gradient norms")
            self._check_norms(grad_norms, "Gradient")
            self.gradient_checks += 1
            
        except Exception as e:
            print(f"   ❌ Exception in gradient monitoring: {str(e)}")
            self.fallback_calls += 1
            # Fallback: just monitor the loss for signs of instability
            print('   🔄 Fallback: monitoring loss only')
            if logs:
                loss_value = logs.get('loss', 0)
                print(f"   📉 Current loss: {loss_value:.2e}")
                if np.isnan(loss_value) or np.isinf(loss_value):
                    print(f"   ⚠️  WARNING: Loss became {loss_value} at batch {batch}")
                elif loss_value > 1e6:
                    print(f"   ⚠️  WARNING: Very large loss {loss_value:.2e} at batch {batch}")
    
    def _check_norms(self, norms, norm_type="Gradient"):
        """Check if norms are within acceptable range"""
        print(f"   🔬 Checking {len(norms)} {norm_type.lower()} norms...")
        warnings = 0
        
        for idx, norm in enumerate(norms):
            try:
                norm_value = float(norm.numpy()) if hasattr(norm, 'numpy') else float(norm)
                
                if norm_value > self.clip_max:
                    print(f"   ⚠️  WARNING: {norm_type} norm {norm_value:.2e} is too large (layer {idx})")
                    warnings += 1
                elif norm_value < self.clip_min:
                    print(f"   ⚠️  WARNING: {norm_type} norm {norm_value:.2e} is too small (layer {idx})")
                    warnings += 1
                elif np.isnan(norm_value) or np.isinf(norm_value):
                    print(f"   ⚠️  WARNING: {norm_type} norm is {norm_value} (layer {idx})")
                    warnings += 1
                    
            except Exception as e:
                print(f"   ❌ Cannot convert norm to float for layer {idx}: {str(e)}")
                continue
                
        if warnings == 0:
            print(f"   ✅ All {norm_type.lower()} norms are within acceptable range")
        else:
            print(f"   ⚠️  Found {warnings} norm warnings")
    
    # def on_epoch_end(self, epoch, logs=None):
    #     """Print summary at end of each epoch"""
    #     print(f"📈 GradientMonitoringCallback: Epoch {epoch + 1} completed")
    #     print(f"   📊 Total batch calls: {self.total_calls}")
    #     print(f"   🔍 Gradient checks performed: {self.gradient_checks}")
    #     print(f"   🔄 Fallback calls: {self.fallback_calls}")
    #     
    #     if logs:
    #         loss = logs.get('loss', 0)
    #         val_loss = logs.get('val_loss', 0)
    #         print(f"   📉 Final epoch loss: {loss:.2e}")
    #         if val_loss:
    #             print(f"   📉 Final epoch val_loss: {val_loss:.2e}")
    #         
    #         if np.isnan(loss) or np.isinf(loss):
    #             print(f"   ⚠️  WARNING: Training loss became unstable: {loss}")
    #         if val_loss and (np.isnan(val_loss) or np.isinf(val_loss)):
    #             print(f"   ⚠️  WARNING: Validation loss became unstable: {val_loss}")
        
    def on_train_end(self, logs=None):
        print("🏁 GradientMonitoringCallback: Training completed!")
        print(f"   📊 Final stats - Total calls: {self.total_calls}, Gradient checks: {self.gradient_checks}, Fallbacks: {self.fallback_calls}")
        
        if self.total_calls == 0:
            print("   ❌ ERROR: Callback was never called! Check if it's properly added to callbacks list.")
        elif self.gradient_checks == 0 and self.fallback_calls == 0:
            print("   ⚠️  WARNING: No gradient monitoring was performed. Check monitor_frequency setting.")
        else:
            print("   ✅ Gradient monitoring completed successfully!")

In [13]:
from keras.src.callbacks import LearningRateScheduler, EarlyStopping, Callback
from keras.src.optimizers import Adam
from keras import Model
import numpy as np


def exponential_decay_schedule(epoch, lr):
    decay_rate = 0.9
    decay_steps = 5
    if epoch % decay_steps == 0 and epoch:
        print('Learning rate update:', lr * decay_rate)
        return lr * decay_rate
    return lr


# Custom callback to monitor LR and stop training
class LRThresholdCallback(Callback):
    def __init__(self, threshold=9e-5):
        super().__init__()
        self.threshold = threshold
        self.should_stop = False

    def on_epoch_end(self, epoch, logs=None):
        lr = float(self.model.optimizer.learning_rate.numpy())
        if lr < self.threshold:
            print(f"\nLearning rate {lr:.6f} < threshold {self.threshold}, moving to next phase.")
            self.model.stop_training = True



In [14]:
class DynamicReduceLROnPlateau(Callback):
    def __init__(self, factor=0.1, patience=3, min_lr=1e-7, verbose=1):
        super().__init__()
        self.factor = factor
        self.patience = patience
        self.min_lr = min_lr
        self.verbose = verbose
        self.best_val_loss = float('inf')
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        current_val_loss = logs.get("val_loss")

        if current_val_loss is None:
            return  # can't do anything if val_loss isn't available

        if current_val_loss < self.best_val_loss - 1e-4:  # a small delta
            self.best_val_loss = current_val_loss
            self.wait = 0
        else:
            self.wait += 1
            if self.wait >= self.patience:
                old_lr = float(self.model.optimizer.learning_rate.numpy())
                if old_lr > self.min_lr:
                    new_lr = max(old_lr * self.factor, self.min_lr)
                    self.model.optimizer.learning_rate.assign(new_lr)
                    if self.verbose:
                        print(f"\nEpoch {epoch+1}: val_loss did not improve. Reducing LR from {old_lr:.6f} to {new_lr:.6f}")
                    self.wait = 0  # reset after LR reduction


In [15]:
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import save_model

class SaveBestModelCallback(Callback):
    def __init__(self, save_path='best_model', monitor='val_loss'):
        super().__init__()
        self.best = float('inf')
        self.monitor = monitor
        self.save_path = save_path
        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        if current is not None and current < self.best:
            self.best = current
            print(f"\nNew best {self.monitor}: {current:.6f}. Saving model...")
            self.model.save(self.save_path+'.keras', overwrite=True)


In [16]:
def train_model(train_data, batch_size=32, validation_split=0.2, Tobs=50, Tpred=60, epochs1=50, epochs2=50, lr1=0.001, lr2=0.000001):
    n_scenarios = train_data.shape[0]
    n_agents = train_data.shape[1]
    X_train_raw = []
    y_train_deltas = []

    # Counters for pruning reasons
    pruned_zero_frame = 0
    pruned_observed_or_future_zero = 0
    total_agents = n_scenarios * n_agents
    
    for i in range(n_scenarios):
        for agent_id in range(2): #foremerly n_agents
            agent_data = train_data[i, agent_id, :, :]  # shape (110, 6)
        
            observed = agent_data[:Tobs]         # shape (Tobs, 6)
            future = agent_data[Tobs:Tobs + Tpred, :2]  # position_x, position_y
            last_obs_pos = observed[-1, :2]
        
            # Skip if more than 20% of observed or future rows are all zeros
            observed_zero_ratio = np.mean(np.all(observed == 0, axis=1))
            future_zero_ratio = np.mean(np.all(future == 0, axis=1))
            
            if observed_zero_ratio > 0.2 or future_zero_ratio > 0.2:
                pruned_observed_or_future_zero += 1
                continue
                
            # Compute deltas w.r.t. previous future timestep
            delta = np.diff(np.vstack([last_obs_pos, future]), axis=0)  # (60, 2)
            
            # prediction of future steps for agent
            X_train_raw.append(observed)
            y_train_deltas.append(delta)
            
    
    # Print pruning summary
    print(f"Total agents: {total_agents}")
    print(f"Pruned due to zero frame in Tobs+Tpred: {pruned_zero_frame}")
    print(f"Pruned due to zero frame in observed or future window: {pruned_observed_or_future_zero}")
    print(f"Remaining valid agents: {len(X_train_raw)}")
    
    
    X_train = np.array(X_train_raw)     # shape (N_valid, Tobs, 6)
    y_train = np.array(y_train_deltas)  # shape (N_valid, Tpred, 2)
    
    # how much of the data is 0?
    # For X_train
    num_elements_X = X_train.size
    num_zeros_X = np.count_nonzero(X_train == 0)
    percent_zeros_X = 100 * num_zeros_X / num_elements_X
    
    # For y_train
    num_elements_y = y_train.size
    num_zeros_y = np.count_nonzero(y_train == 0)
    percent_zeros_y = 100 * num_zeros_y / num_elements_y
    
    print(f"X_train: {num_zeros_X} zeros out of {num_elements_X} elements ({percent_zeros_X:.2f}%)")
    print(f"y_train: {num_zeros_y} zeros out of {num_elements_y} elements ({percent_zeros_y:.2f}%)")
    
    # print(f"ex. y_train {y_train[0]}")


    print(f"Training on {X_train.shape[0]} valid agent trajectories.")
    print(f"Input shape: {X_train.shape}, Delta Output shape: {y_train.shape}")
    
     # --- Normalize Input and Output ---
    X_mean = X_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 6)
    X_std = X_train.std(axis=(0, 1), keepdims=True) + 1e-8

    y_mean = y_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 2)
    y_std = y_train.std(axis=(0, 1), keepdims=True) + 1e-8

    X_train = (X_train - X_mean) / X_std
    y_train = (y_train - y_mean) / y_std
    
    # X_mean, X_std, y_mean, y_std = None, None, None, None
    
    
    # print(X_train[:2])
    # print(y_train[:2])
    
    model = create_lstm_encoder_decoder_with_attention(
        input_dim=X_train.shape[-1],
        output_dim=2,
        timesteps_in=Tobs,
        timesteps_out=Tpred,
        loss_fn='mse',
        lr=lr1
    )
    
    gradient_monitoring_callback = GradientMonitoringCallback(clip_min=1e-4, clip_max=1e2)
    
    # Pass normalization parameters to SaveBestModelCallback
    save_best_callback = SaveBestModelCallback(
        save_path='lstm2', 
        monitor='val_loss',
    )


    phase1_callbacks = [
        # LearningRateScheduler(exponential_decay_schedule),
        DynamicReduceLROnPlateau(factor=0.7, patience=3, min_lr=1e-9),
        EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss'),
        LRThresholdCallback(threshold=9e-8),
        # gradient_monitoring_callback,
        save_best_callback
    ]

    print("\n--- Phase 1: Training ---")
    model.fit(
        X_train, y_train,
        epochs=epochs1,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=phase1_callbacks,
        verbose=1
    )

    print("\n--- Phase 2: Fine-tuning ---")
    model.compile(
        optimizer=Adam(
            learning_rate=lr2,
            clipnorm=0.1,      # More aggressive clipping
            beta_1=0.9,         # Standard momentum
            beta_2=0.999,       # Standard RMSprop decay
            epsilon=1e-7        # Smaller epsilon for stability
        ),
        loss='mse',
        metrics=['mae']
    )
    phase2_callbacks = [
        # LearningRateScheduler(exponential_decay_schedule),
        EarlyStopping(patience=3, restore_best_weights=True, monitor='val_loss'), 
        DynamicReduceLROnPlateau(factor=0.5, patience=2, min_lr=1e-9),
        LRThresholdCallback(threshold=9e-8),
        # gradient_monitoring_callback
    ]
    
    model.fit(
        X_train, y_train,
        epochs=epochs2,
        batch_size=batch_size,
        validation_split=validation_split,
        callbacks=phase2_callbacks,
        verbose=1
    )
    
    print(f"X_mean:{X_mean}, X_std:{X_std}, y_mean:{y_mean}, y_std:{y_std}")

    # Return model and normalization parameters
    return model, X_mean, X_std, y_mean, y_std

In [17]:
def plot_mae_by_timestep(y_true, y_pred):
    """
    Visualize MAE across timesteps in the prediction horizon.
    
    Args:
        y_true (np.ndarray): shape (N, Tpred, 2)
        y_pred (np.ndarray): shape (N, Tpred, 2)
    """
    mae_per_timestep = np.mean(np.abs(y_true - y_pred), axis=(0, 2))  # shape (Tpred,)
    
    import matplotlib.pyplot as plt
    plt.figure(figsize=(10, 4))
    plt.plot(mae_per_timestep, label='MAE per Timestep')
    plt.xlabel('Timestep')
    plt.ylabel('MAE (meters)')
    plt.title('Mean Absolute Error Over Prediction Horizon')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [16]:
import numpy as np

def reconstruct_absolute_positions(pred_deltas, last_observed_positions):
    """
    Reconstruct absolute predicted positions in an autoregressive way.
    
    Args:
        pred_deltas: np.ndarray of shape (N, Tpred, 2)
        last_observed_positions: np.ndarray of shape (N, 2)
    
    Returns:
        np.ndarray of shape (N, Tpred, 2)
    """
    return last_observed_positions[:, None, :] + np.cumsum(pred_deltas, axis=1)


In [17]:
def forecast_positions(scenario_data, Tobs, Tpred, model, X_mean=None, X_std=None, y_mean=None, y_std=None):
    """
    Use LSTM model to forecast future deltas and reconstruct absolute positions of only the ego
    Applies normalization only if statistics are provided.

    Args:
        scenario_data (numpy.ndarray): Shape (agents, time_steps, dimensions)
        Tobs (int): Number of observed time steps
        Tpred (int): Number of future time steps to predict
        model (Model): Trained LSTM model
        X_mean, X_std: Optional normalization stats for input
        y_mean, y_std: Optional normalization stats for output

    Returns:
        numpy.ndarray: Predicted absolute positions of shape (Tpred, 2)
    """
    
    # Only process ego agent (agent_index=0)
    agent_idx = 0
    agent_data = scenario_data[agent_idx, :Tobs, :].copy()  # shape (Tobs, 6)
    
    # Skip if fully padded
    if np.all(agent_data == 0):
        return np.zeros((Tpred, 2))
    
    X_pred = np.expand_dims(agent_data, axis=0)  # shape (1, Tobs, 6)

    # Normalize if stats are provided
    if X_mean is not None and X_std is not None:
        X_pred = (X_pred - X_mean) / X_std

    # Predict deltas (normalized or raw)
    pred_deltas = model.predict(X_pred, verbose=0)  # shape (1, Tpred, 2)
    
    # print("pred deltas")
    # print(pred_deltas[:,:])

    # Denormalize if stats are provided
    if y_mean is not None and y_std is not None:
        pred_deltas = pred_deltas * y_std + y_mean

    # Reconstruct absolute positions
    last_pos = agent_data[Tobs - 1, :2]  # shape (2,)
    abs_positions = reconstruct_absolute_positions(
        pred_deltas=pred_deltas,
        last_observed_positions=np.expand_dims(last_pos, axis=0)
    )[0]
    
    return abs_positions

In [11]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def make_gif(data_matrix1, data_matrix2, name='comparison'):
    import numpy as np
    import matplotlib.pyplot as plt
    import matplotlib.animation as animation

    cmap1 = plt.cm.get_cmap('viridis', 50)
    cmap2 = plt.cm.get_cmap('plasma', 50)

    assert data_matrix1.shape[1] == data_matrix2.shape[1], "Both matrices must have same number of timesteps"
    timesteps = data_matrix1.shape[1]

    fig, axes = plt.subplots(1, 2, figsize=(18, 9))
    ax1, ax2 = axes

    def update(frame):
        for ax in axes:
            ax.clear()

        for i in range(data_matrix1.shape[0]):
            for (data_matrix, ax, cmap) in [(data_matrix1, ax1, cmap1), (data_matrix2, ax2, cmap2)]:
                x = data_matrix[i, frame, 0]
                y = data_matrix[i, frame, 1]
                if x != 0 and y != 0:
                    xs = data_matrix[i, :frame+1, 0]
                    ys = data_matrix[i, :frame+1, 1]
                    mask = (xs != 0) & (ys != 0)
                    xs = xs[mask]
                    ys = ys[mask]
                    if len(xs) > 0 and len(ys) > 0:
                        color = cmap(i)
                        ax.plot(xs, ys, alpha=0.9, color=color)
                        ax.scatter(x, y, s=80, color=color)

        # Plot ego vehicle (index 0) on both
        ax1.plot(data_matrix1[0, :frame, 0], data_matrix1[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
        ax1.scatter(data_matrix1[0, frame, 0], data_matrix1[0, frame, 1], s=80, color='tab:orange')
        ax1.set_title('Prediction')

        ax2.plot(data_matrix2[0, :frame, 0], data_matrix2[0, :frame, 1], color='tab:orange', label='Ego Vehicle')
        ax2.scatter(data_matrix2[0, frame, 0], data_matrix2[0, frame, 1], s=80, color='tab:orange')
        ax2.set_title('Actual')

        for ax, data_matrix in zip(axes, [data_matrix1, data_matrix2]):
            ax.set_xlim(data_matrix[:, :, 0][data_matrix[:, :, 0] != 0].min() - 10,
                        data_matrix[:, :, 0][data_matrix[:, :, 0] != 0].max() + 10)
            ax.set_ylim(data_matrix[:, :, 1][data_matrix[:, :, 1] != 0].min() - 10,
                        data_matrix[:, :, 1][data_matrix[:, :, 1] != 0].max() + 10)
            ax.legend()
            ax.set_xlabel('X')
            ax.set_ylabel('Y')

        # Compute MSE over non-zero entries up to current frame
        mask = (data_matrix2[:, :frame+1, :] != 0) & (data_matrix1[:, :frame+1, :] != 0)
        mse = np.mean((data_matrix1[:, :frame+1, :][mask] - data_matrix2[:, :frame+1, :][mask]) ** 2)

        fig.suptitle(f"Timestep {frame} - MSE: {mse:.4f}", fontsize=16)
        return ax1.collections + ax1.lines + ax2.collections + ax2.lines

    anim = animation.FuncAnimation(fig, update, frames=list(range(0, timesteps, 3)), interval=100, blit=True)
    anim.save(f'trajectory_visualization_{name}.gif', writer='pillow')
    plt.close()


In [13]:
# figure out stats
def stats():
    Tobs = 50
    Tpred = 60
    n_scenarios = standardized_train_data.shape[0]
    X_train_raw = []
    y_train_deltas = []

    for i in range(n_scenarios):
        ego_data = standardized_train_data[i, 0, :, :]
        if np.all(ego_data == 0):
            continue

        observed = ego_data[:Tobs]            # shape (50, 6)
        future = ego_data[Tobs:Tobs+Tpred, :2]
        last_obs_pos = observed[-1, :2]

        if np.any(np.all(observed == 0, axis=1)) or np.any(np.all(future == 0, axis=1)):
            continue

        # Compute deltas w.r.t. previous future timestep
        delta = np.diff(np.vstack([last_obs_pos, future]), axis=0)  # (60, 2)

        X_train_raw.append(observed)
        y_train_deltas.append(delta)


    X_train = np.array(X_train_raw)
    y_train = np.array(y_train_deltas)

    print(f"{X_train.shape[0]} valid sequences.")
    print(f"Input shape: {X_train.shape}, Delta Output shape: {y_train.shape}")

    # --- Normalize Input and Output ---
    X_mean = X_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 6)
    X_std = X_train.std(axis=(0, 1), keepdims=True) + 1e-8

    y_mean = y_train.mean(axis=(0, 1), keepdims=True)  # shape: (1, 1, 2)
    y_std = y_train.std(axis=(0, 1), keepdims=True) + 1e-8

    X_train = (X_train - X_mean) / X_std
    y_train = (y_train - y_mean) / y_std
    return X_mean, X_std, y_mean, y_std
X_mean, X_std, y_mean, y_std = stats()

9998 valid sequences.
Input shape: (9998, 50, 6), Delta Output shape: (9998, 60, 2)


In [22]:
K.clear_session()

In [23]:
model = create_lstm_encoder_decoder_with_attention(10000,2,50,60)

2025-05-29 22:56:27.291621: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9804 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:60:00.0, compute capability: 7.5


In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50, 10000)]          0         []                            
                                                                                                  
 encoder_lstm_0 (LSTM)       (None, 50, 512)              2153062   ['input_1[0][0]']             
                                                          4                                       
                                                                                                  
 encoder_lstm_1 (LSTM)       (None, 50, 512)              2099200   ['encoder_lstm_0[0][0]']      
                                                                                                  
 encoder_lstm_2 (LSTM)       (None, 50, 512)              2099200   ['encoder_lstm_1[0][0]']  

In [25]:
# fit
model, X_mean, X_std, y_mean, y_std = train_model(standardized_train_data[:],epochs1=50, epochs2=10, validation_split=0.2, lr1=0.001, lr2=0.00001)

Total agents: 500000
Pruned due to zero frame in Tobs+Tpred: 0
Pruned due to zero frame in observed or future window: 1
Remaining valid agents: 19999
X_train: 947758 zeros out of 5999700 elements (15.80%)
y_train: 0 zeros out of 2399880 elements (0.00%)
Training on 19999 valid agent trajectories.
Input shape: (19999, 50, 6), Delta Output shape: (19999, 60, 2)
🔧 GradientMonitoringCallback initialized with clip_min=0.0001, clip_max=100.0, monitor_freq=3

--- Phase 1: Training ---
Epoch 1/50


W0000 00:00:1748559398.787330 3127847 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 2080 Ti" frequency: 1545 num_cores: 68 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5767168 shared_memory_size_per_multiprocessor: 65536 memory_size: 10280763392 bandwidth: 616000000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
2025-05-29 22:56:39.438326: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902


500/500 [==============================] - ETA: 0s - loss: 0.2270 - mae: 0.3035

W0000 00:00:1748559424.567212 3127847 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 2080 Ti" frequency: 1545 num_cores: 68 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5767168 shared_memory_size_per_multiprocessor: 65536 memory_size: 10280763392 bandwidth: 616000000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }



New best val_loss: 0.163348. Saving model...
500/500 [==============================] - 35s 53ms/step - loss: 0.2270 - mae: 0.3035 - val_loss: 0.1633 - val_mae: 0.2464
Epoch 2/50
499/500 [============================>.] - ETA: 0s - loss: 0.1476 - mae: 0.2354
New best val_loss: 0.149131. Saving model...
500/500 [==============================] - 25s 49ms/step - loss: 0.1475 - mae: 0.2354 - val_loss: 0.1491 - val_mae: 0.2301
Epoch 3/50
499/500 [============================>.] - ETA: 0s - loss: 0.1299 - mae: 0.2176
New best val_loss: 0.122220. Saving model...
500/500 [==============================] - 25s 50ms/step - loss: 0.1298 - mae: 0.2176 - val_loss: 0.1222 - val_mae: 0.2121
Epoch 4/50
500/500 [==============================] - ETA: 0s - loss: 0.1183 - mae: 0.2060
New best val_loss: 0.120806. Saving model...
500/500 [==============================] - 25s 50ms/step - loss: 0.1183 - mae: 0.2060 - val_loss: 0.1208 - val_mae: 0.2091
Epoch 5/50
499/500 [============================>.] - 

W0000 00:00:1748560186.745032 3127847 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 2080 Ti" frequency: 1545 num_cores: 68 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5767168 shared_memory_size_per_multiprocessor: 65536 memory_size: 10280763392 bandwidth: 616000000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


500/500 [==============================] - ETA: 0s - loss: 0.0616 - mae: 0.1394

W0000 00:00:1748560212.992463 3127847 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 2080 Ti" frequency: 1545 num_cores: 68 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5767168 shared_memory_size_per_multiprocessor: 65536 memory_size: 10280763392 bandwidth: 616000000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


500/500 [==============================] - 35s 56ms/step - loss: 0.0616 - mae: 0.1394 - val_loss: 0.0793 - val_mae: 0.1545
Epoch 2/10
500/500 [==============================] - 27s 54ms/step - loss: 0.0599 - mae: 0.1358 - val_loss: 0.0791 - val_mae: 0.1538
Epoch 3/10
500/500 [==============================] - 27s 54ms/step - loss: 0.0591 - mae: 0.1346 - val_loss: 0.0790 - val_mae: 0.1534
Epoch 4/10
500/500 [==============================] - ETA: 0s - loss: 0.0585 - mae: 0.1338
Epoch 4: val_loss did not improve. Reducing LR from 0.000010 to 0.000005
500/500 [==============================] - 27s 54ms/step - loss: 0.0585 - mae: 0.1338 - val_loss: 0.0790 - val_mae: 0.1532
Epoch 5/10
500/500 [==============================] - 26s 53ms/step - loss: 0.0580 - mae: 0.1331 - val_loss: 0.0791 - val_mae: 0.1532
Epoch 6/10
500/500 [==============================] - ETA: 0s - loss: 0.0578 - mae: 0.1329
Epoch 6: val_loss did not improve. Reducing LR from 0.000005 to 0.000002
500/500 [===============

In [26]:
model = load_model()

Keras model loaded from lstm2.keras


In [27]:
# visualize regular prediction

# model = load_model()

# Parameters
Tobs = 50
Tpred = 60

data = standardized_train_data[6325]

# Select a test scenario (can use any valid index)
test_scenario = data.copy()  # shape (agents, time_steps, features)


# Forecast future positions
predicted_positions = forecast_positions(test_scenario, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std)

# Create combined matrix of past observed + predicted for ego agent (agent 0)
ego_past = test_scenario[0, :Tobs, :2]               # shape (Tobs, 2)
ego_future = predicted_positions                     # shape (Tpred, 2)

print(ego_future[:5])
print(test_scenario[0, Tobs:Tobs+5, :2])
ego_full = np.concatenate([ego_past, ego_future], axis=0)  # shape (Tobs + Tpred, 2)

# Create updated scenario with predicted ego and original others
updated_scenario = test_scenario.copy()
updated_scenario[0, :Tobs+Tpred, :2] = ego_full  # Replace ego trajectory

# Visualize
make_gif(updated_scenario, data, name='lstm2')

W0000 00:00:1748560380.343540 3127847 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "GPU" vendor: "NVIDIA" model: "NVIDIA GeForce RTX 2080 Ti" frequency: 1545 num_cores: 68 environment { key: "architecture" value: "7.5" } environment { key: "cuda" value: "12020" } environment { key: "cudnn" value: "8904" } num_registers: 65536 l1_cache_size: 24576 l2_cache_size: 5767168 shared_memory_size_per_multiprocessor: 65536 memory_size: 10280763392 bandwidth: 616000000 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }


[[ 19.72889216 144.68795664]
 [ 19.71153917 144.23530889]
 [ 19.69581045 143.79343459]
 [ 19.67851866 143.3536886 ]
 [ 19.66052052 142.92034789]]
[[ 19.71302446 144.75582367]
 [ 19.68203702 144.34848456]
 [ 19.65202753 143.95338695]
 [ 19.62305751 143.57145674]
 [ 19.59515847 143.20213145]]


/tmp/ipykernel_3127847/849000278.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap1 = plt.cm.get_cmap('viridis', 50)
/tmp/ipykernel_3127847/849000278.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap2 = plt.cm.get_cmap('plasma', 50)


In [28]:
# # visualize prediction
# 
# # model = load_model()
# 
# # Parameters
# Tobs = 50
# Tpred = 60
# 
# data = train_data[0]
# 
# # Select a test scenario (can use any valid index)
# test_scenario = data.copy()  # shape (agents, time_steps, features)
# 
# 
# # Forecast future positions
# predicted_positions = finetune_forecast_positions(test_scenario, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std)
# 
# # Create combined matrix of past observed + predicted for ego agent (agent 0)
# ego_past = test_scenario[0, :Tobs, :2]               # shape (Tobs, 2)
# ego_future = predicted_positions[0]                  # shape (Tpred, 2)
# ego_full = np.concatenate([ego_past, ego_future], axis=0)  # shape (Tobs + Tpred, 2)
# 
# # Create updated scenario with predicted ego and original others
# updated_scenario = test_scenario.copy()
# updated_scenario[0, :Tobs+Tpred, :2] = ego_full  # Replace ego trajectory
# 
# # Visualize
# make_gif(updated_scenario, data, name='lstm2')

In [14]:
from sklearn.metrics import mean_squared_error


def evaluate_mse(train_data, model, Tobs=50, Tpred=60):
    """
    Computes LSTM prediction for ego agent and evaluates MSE with progress reporting.
    """
    N = train_data.shape[0]
    mse_list = []
    valid_scenarios = 0
    
    print(f"Evaluating {N} scenarios...")
    
    # Progress reporting variables
    report_interval = max(1, N // 10)  # Report at 10% intervals
    
    for i in range(N):
        # Progress reporting
        if i % report_interval == 0 or i == N-1:
            print(f"Processing scenario {i+1}/{N} ({(i+1)/N*100:.1f}%)")
        
        scenario_data = train_data[i]
        ego_agent_data = scenario_data[0]
        ground_truth = ego_agent_data[Tobs:Tobs+Tpred, :2]
        
        # Skip if ground truth contains all zeros (padded)
        if np.all(ground_truth == 0):
            continue
            
        valid_scenarios += 1
        
        # Forecast future positions
        predicted_positions = forecast_positions(
            ego_agent_data[np.newaxis, :, :],
            Tobs, Tpred, model, X_mean, X_std, y_mean, y_std
        )
        
        # Compute MSE
        mse = mean_squared_error(ground_truth, predicted_positions)
        mse_list.append(mse)
        
        # Occasional MSE reporting
        if i % report_interval == 0:
            print(f"  Current scenario MSE: {mse:.4f}")
    
    # Final results
    if mse_list:
        overall_mse = np.mean(mse_list)
        print(f"Evaluation complete: {valid_scenarios} valid scenarios")
        print(f"Mean Squared Error (MSE): {overall_mse:.4f}")
        print(f"Min MSE: {np.min(mse_list):.4f}, Max MSE: {np.max(mse_list):.4f}")
        return overall_mse
    else:
        print("No valid scenarios for evaluation.")
        return None

In [30]:
# Evaluate on training data
evaluate_mse(standardized_train_data, model)

Evaluating 10000 scenarios...
Processing scenario 1/10000 (0.0%)
  Current scenario MSE: 3.0316
Processing scenario 1001/10000 (10.0%)
  Current scenario MSE: 1.6629
Processing scenario 2001/10000 (20.0%)
  Current scenario MSE: 0.0797
Processing scenario 3001/10000 (30.0%)
  Current scenario MSE: 2.5753
Processing scenario 4001/10000 (40.0%)
  Current scenario MSE: 0.3912
Processing scenario 5001/10000 (50.0%)
  Current scenario MSE: 29.0094
Processing scenario 6001/10000 (60.0%)
  Current scenario MSE: 29.8110
Processing scenario 7001/10000 (70.0%)
  Current scenario MSE: 17.9062
Processing scenario 8001/10000 (80.0%)
  Current scenario MSE: 7.8313
Processing scenario 9001/10000 (90.0%)
  Current scenario MSE: 0.1848
Processing scenario 10000/10000 (100.0%)
Evaluation complete: 10000 valid scenarios
Mean Squared Error (MSE): 8.4197
Min MSE: 0.0002, Max MSE: 239.4549


8.419701296160717

In [18]:
import pandas as pd
import numpy as np

def generate_submission(data, output_csv, model, Tobs=50, Tpred=60, X_mean=None, X_std=None, y_mean=None, y_std=None):
    """
    Applies standardization, forecasting, and de-standardization to generate
    a submission CSV in the format: index,x,y
    
    Args:
        data (np.ndarray): Test data of shape (num_scenarios, 50, 110, 6)
        output_csv (str): Output CSV file path
        model: Trained LSTM model
        Tobs (int): Number of observed time steps
        Tpred (int): Number of time steps to predict
        X_mean, X_std: Optional normalization stats for input
        y_mean, y_std: Optional normalization stats for output
    """
    predictions = []
    
    for i in range(data.shape[0]):
        scenario_data = data[i]  # Shape: (50, 110, 6)
        
        # Standardize coordinates
        standardized_data, min_values = standardize_data_dimensions(scenario_data)
        
        # Extract ego agent data (agent 0) - transpose to get correct shape
        ego_agent_data = standardized_data[:, 0, :]  # Shape: (50, 6)
        ego_agent_data = ego_agent_data[np.newaxis, :, :]  # Shape: (1, 50, 6)
        
        # Predict future positions for the ego agent
        predicted_positions = forecast_positions(
            ego_agent_data, Tobs, Tpred, model, X_mean, X_std, y_mean, y_std
        )  # Shape: (60, 2)
        
        # De-standardize predicted positions
        predicted_positions[:, 0] += min_values[0]  # x
        predicted_positions[:, 1] += min_values[1]  # y
        
        # Append to all predictions
        predictions.extend(predicted_positions)
    
    # Create submission DataFrame
    submission_df = pd.DataFrame(predictions, columns=["x", "y"])
    submission_df.index.name = 'index'
    submission_df.to_csv(output_csv)
    print(f"Submission file '{output_csv}' saved with shape {submission_df.shape}")

# Example usage:
generate_submission(test_data, 'lstm_attention_submission.csv', model, 
                   X_mean=X_mean, X_std=X_std, y_mean=y_mean, y_std=y_std)

Submission file 'lstm_attention_submission.csv' saved with shape (126000, 2)
